In [2]:
!pip install apache-airflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.3/511.3 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 7.4 MB/

In [4]:
!pip install apache-airflow[postgres]==<desired_version>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install apache-airflow[postgres]==<desired_version>'


In [6]:
!pip install --upgrade apache-airflow

In [8]:
!pip install apache-airflow-providers-postgres

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.9 MB/s eta 0:00:00


In [9]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime
from airflow.hooks.postgres_hook import PostgresHook
import requests

<ipython-input-9-f2b92efcee94>:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

<ipython-input-9-f2b92efcee94>:4 DeprecationWarning: The `airflow.hooks.postgres_hook.PostgresHook` class is deprecated. Please use `'airflow.providers.postgres.hooks.postgres.PostgresHook'`.

In [12]:
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

<ipython-input-12-b2468d4201df>:1 DeprecationWarning: The `airflow.operators.dummy_operator.DummyOperator` class is deprecated. Please use `'airflow.operators.empty.EmptyOperator'`.

<ipython-input-12-b2468d4201df>:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [14]:
db_connection = {
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'user': 'postgres',
    'password': 'bunga123'
}

In [19]:
def extract_data():
    url = "https://api.openaq.org/v2/locations?limit=100&page=1&offset=0&sort=desc&radius=1000&country=ID&order_by=lastUpdated&dump_raw=false"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    print(response.text)

    jakarta_data = [item for item in data['results'] if item['country'] == 'ID' and item['name'] == 'Jakarta Central']

    return jakarta_data

In [22]:
def save_to_database(**kwargs):
    data = kwargs['ti'].xcom_pull(task_ids='extract_data')

    # Koneksi ke database menggunakan Hook dengan informasi koneksi yang telah ditentukan
    postgres_hook = PostgresHook(postgres_conn_id='Kualitas_udara_connection', conn_name_attr=db_conn)

    # Membuat tabel 'openaq_data'
    create_table_query = """
    CREATE TABLE IF NOT EXISTS openaq_data (
        id SERIAL PRIMARY KEY,
        city TEXT,
        name TEXT,
        entity TEXT,
        country TEXT,
        sources TEXT,
        isMobile BOOLEAN,
        isAnalysis BOOLEAN,
        parameters JSONB,
        sensorType TEXT,
        coordinates JSONB,
        lastUpdated TIMESTAMP,
        firstUpdated TIMESTAMP,
        measurements INTEGER,
        bounds JSONB,
        manufacturers JSONB
    );
    """
    postgres_hook.run(create_table_query)

In [24]:
def save_to_database(**kwargs):
    # ...
    for item in data:
        insert_query = """
        INSERT INTO openaq_data (city, name, entity, country, sources, isMobile, isAnalysis, parameters, sensorType, coordinates, lastUpdated, firstUpdated, measurements, bounds, manufacturers)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        postgres_hook.run(insert_query, parameters=(
            item['city'],
            item['name'],
            item['entity'],
            item['country'],
            item['sources'],
            item['isMobile'],
            item['isAnalysis'],
            item['parameters'],
            item['sensorType'],
            item['coordinates'],
            item['lastUpdated'],
            item['firstUpdated'],
            item['measurements'],
            item['bounds'],
            item['manufacturers']
        ))


In [25]:
dag = DAG(
    'openaq_data_pipeline',
    start_date=datetime(2023, 10, 21, 4, 0),  # Jadwalkan eksekusi pada jam 4 pagi
    schedule_interval='0 4 * * *',  # Ekspresi cron
    catchup=False,  # Jangan mengejar eksekusi yang terlewat
)

<ipython-input-25-8a35c28eeddb>:1 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

In [26]:
extract_task = PythonOperator(
    task_id='extract_data',
    python_callable=extract_data,
    dag=dag,
)

In [27]:
save_to_db_task = PythonOperator(
    task_id='save_to_database',
    python_callable=save_to_database,
    provide_context=True,
    dag=dag,
)

<ipython-input-27-a4b15ed652d0>:1 RemovedInAirflow3Warning: provide_context is deprecated as of 2.0 and is no longer required

In [28]:
extract_task >> save_to_db_task

<Task(PythonOperator): save_to_database>